In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x:'%.2f'%x)
pd.set_option('display.max_rows', 300)
sns.set(rc={'figure.figsize':(12,8.5)})
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()


from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [ ]:
df = pd.read_csv('renewable.csv')
df

In [ ]:
def ranking():
    print('Choose from a range of years 1994 to 2014')
    start = ''
    end = ''
    country = ''
    while start == '':
        start = int(input('Enter the initial year: '))
        end = int(input('Enter the final year: '))
        country = input('Enter the country: ')
        if start == end:
            df1 = df[df['year'] == start]
            df1['rank'] = df1['new_quantity'].rank(method='dense', ascending = False)
            place = df1[df1['country_or_area'] == country]
            num = df1[df1['country_or_area'] == country]['rank'].values[0]

            three_first = df1[df1['rank']<4].sort_values('rank')
            three_last = df1[df1['rank']>(df1['rank'].max()-3)].sort_values('rank')

            txt = 'Your chosen country {} was ranked #{} in the year {}.'.format(country,int(num), start)
            print(txt)
            place.iplot(kind = 'bar', x= 'country_or_area', y=['new_quantity','non_renewable','renewable'])

            print('3 first countries: ', three_first['country_or_area'].values[[0,1,2]])
            three_first.iplot(kind = 'bar', x= 'country_or_area', y=['new_quantity','non_renewable','renewable'])

            print('3 last countries: ',  three_last['country_or_area'].values[[0,1,2]])
            three_last.iplot(kind = 'bar', x= 'country_or_area', y=['new_quantity','non_renewable','renewable'])

        else:
            df1 = df[(df['year'] >= start) & (df['year'] <= end)]
            df1 = df1.groupby('country_or_area').agg({'new_quantity':'mean','renewable':'mean','non_renewable':'mean'}).reset_index()
            df1['rank'] = df1['new_quantity'].rank(method='dense', ascending = False)
            place = df1[df1['country_or_area'] == country]
            num = df1[df1['country_or_area'] == country]['rank'].values[0]

            three_first = df1[df1['rank']<4].sort_values('rank')
            three_last = df1[df1['rank']>(df1['rank'].max()-3)].sort_values('rank')

            txt = 'Your chosen country {} was ranked #{} in the years {} - {}.'.format(country,int(num),start, end)
            print(txt)
            place.iplot(kind = 'bar', x= 'country_or_area', y=['new_quantity','non_renewable','renewable'])

            print('3 first countries: ', three_first['country_or_area'].values[[0,1,2]])
            three_first.iplot(kind = 'bar', x= 'country_or_area', y=['new_quantity','non_renewable','renewable'])

            print('3 last countries: ',  three_last['country_or_area'].values[[0,1,2]])
            three_last.iplot(kind = 'bar', x= 'country_or_area', y=['new_quantity','non_renewable','renewable'])

